In [18]:
import xgboost as xgb
import shap
import pandas as pd
from pathlib import Path
import os
import numpy as np
from src.decision_tree.shap_wrapper import ShapWrapper

In [2]:
while "notebooks" in os.getcwd():
    os.chdir("../")

In [3]:
model_xgb = xgb.Booster()
model_xgb.load_model("models/winequality_red_saved.json")

In [4]:
DATA_DIR = Path('data/wine_quality')
file_name = 'winequality_red.csv'
scaled_file_name = 'winequality_red_scaled.csv'
train_file_name = 'train_winequality_red_scaled.csv'
test_file_name = 'test_winequality_red_scaled.csv'

In [5]:
df = pd.read_csv(DATA_DIR / test_file_name, sep=',')
df.info()
#df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         320 non-null    float64
 1   volatile_acidity      320 non-null    float64
 2   citric_acid           320 non-null    float64
 3   residual_sugar        320 non-null    float64
 4   chlorides             320 non-null    float64
 5   free_sulfur_dioxide   320 non-null    float64
 6   total_sulfur_dioxide  320 non-null    float64
 7   density               320 non-null    float64
 8   pH                    320 non-null    float64
 9   sulphates             320 non-null    float64
 10  alcohol               320 non-null    float64
 11  quality               320 non-null    int64  
dtypes: float64(11), int64(1)
memory usage: 30.1 KB


In [6]:

X = df.loc[:, df.columns != 'quality']
y = df.loc[:, df.columns == 'quality']

In [21]:
X = X.iloc[0:1, :]

In [22]:
sh = ShapWrapper()
rank = sh.get_shap_ranking(model_xgb, X)

[11:34:35] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


In [25]:
rank[0]

['residual_sugar',
 'pH',
 'fixed_acidity',
 'citric_acid',
 'free_sulfur_dioxide',
 'chlorides',
 'density',
 'total_sulfur_dioxide',
 'sulphates',
 'alcohol',
 'volatile_acidity']